In [1]:
import numpy as np
import tensorflow as tf
import math
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.insert(0, '/home/todor/emotion_recognition/facenet')
from facenet.src.load_model import *

In [2]:
X_train2 = np.load('data/SFEW_2/Train/train_aug2.npy')
y_train2 = np.load('data/SFEW_2/Train/train_aug2_labels.npy')
X_train10 = np.load('data/SFEW_2/Train/train_aug10.npy')
y_train10 = np.load('data/SFEW_2/Train/train_aug10_labels.npy')
X_val = np.load('data/SFEW_2/Val/val.npy')
y_val = np.load('data/SFEW_2/Val/val_labels.npy')

X_train2 = X_train2.astype(np.float32)
X_train10 = X_train10.astype(np.float32)
X_val = X_val.astype(np.float32)

print X_train2.shape
print X_train10.shape
print X_val.shape
print y_val.shape

(1751, 160, 160, 3)
(9801, 160, 160, 3)
(431, 160, 160, 3)
(431,)


In [4]:
load_model('facenet/pre-trained/20170511-185253.pb')
X = tf.get_default_graph().get_tensor_by_name('input:0')
phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
fno1 = tf.get_default_graph().get_tensor_by_name('InceptionResnetV1/Logits/Dropout/cond/Merge:0')
fno2 = tf.get_default_graph().get_tensor_by_name('InceptionResnetV1/Bottleneck/BatchNorm/batchnorm/add_1:0')
fno3 = tf.get_default_graph().get_tensor_by_name('embeddings:0')

o1_sz = fno1.get_shape()[1]
o2_sz = fno2.get_shape()[1]
o3_sz = fno3.get_shape()[1]




t2_out1 = np.zeros((1751, o1_sz))
t2_out2 = np.zeros((1751, o2_sz))
t2_out3 = np.zeros((1751, o3_sz))
t10_out1 = np.zeros((9801, o1_sz))
t10_out2 = np.zeros((9801, o2_sz))
t10_out3 = np.zeros((9801, o3_sz))
v_out1 = np.zeros((431, o1_sz))
v_out2 = np.zeros((431, o2_sz))
v_out3 = np.zeros((431, o3_sz))

batch_size = 100

for i in xrange(X_train2.shape[0] / batch_size + 1):
    print 'Data2: ' + str(i)
    si = i*batch_size
    ei = min((i+1)*batch_size, X_train2.shape[0])
    feed_dict2 = {X: X_train2[si:ei,:,:,:],
                  phase_train_placeholder: False}
    t2_out1[si:ei,:], t2_out2[si:ei,:], t2_out3[si:ei,:] = tf.Session().run([fno1, fno2, fno3],
                                                                            feed_dict=feed_dict2)
    
for i in xrange(X_train10.shape[0] / batch_size + 1):
    print 'Data10: ' + str(i)
    si = i*batch_size
    ei = min((i+1)*batch_size, X_train10.shape[0])
    feed_dict10 = {X: X_train10[si:ei,:,:,:], 
                   phase_train_placeholder: False}
    t10_out1[si:ei,:], t10_out2[si:ei,:], t10_out3[si:ei,:] = tf.Session().run([fno1, fno2, fno3],
                                                                               feed_dict=feed_dict10)
    
for i in xrange(X_val.shape[0] / batch_size + 1):
    print 'Val: ' + str(i)
    si = i*batch_size
    ei = min((i+1)*batch_size, X_val.shape[0])
    feed_dict_val = {X: X_val[si:ei,:,:,:],
                     phase_train_placeholder: False}
    v_out1[si:ei,:], v_out2[si:ei,:], v_out3[si:ei,:] = tf.Session().run([fno1, fno2, fno3],feed_dict=feed_dict_val)

train2_out = np.concatenate((t2_out1, t2_out2, t2_out3), axis=1)
train10_out = np.concatenate((t10_out1, t10_out2, t10_out3), axis=1)
val_out = np.concatenate((v_out1, v_out2, v_out3), axis=1)


Data2: 0
Data2: 1
Data2: 2
Data2: 3
Data2: 4
Data2: 5
Data2: 6
Data2: 7
Data2: 8
Data2: 9
Data2: 10
Data2: 11
Data2: 12
Data2: 13
Data2: 14
Data2: 15
Data2: 16
Data2: 17
Data10: 0
Data10: 1
Data10: 2
Data10: 3
Data10: 4
Data10: 5
Data10: 6
Data10: 7
Data10: 8
Data10: 9
Data10: 10
Data10: 11
Data10: 12
Data10: 13
Data10: 14
Data10: 15
Data10: 16
Data10: 17
Data10: 18
Data10: 19
Data10: 20
Data10: 21
Data10: 22
Data10: 23
Data10: 24
Data10: 25
Data10: 26
Data10: 27
Data10: 28
Data10: 29
Data10: 30
Data10: 31
Data10: 32
Data10: 33
Data10: 34
Data10: 35
Data10: 36
Data10: 37
Data10: 38
Data10: 39
Data10: 40
Data10: 41
Data10: 42
Data10: 43
Data10: 44
Data10: 45
Data10: 46
Data10: 47
Data10: 48
Data10: 49
Data10: 50
Data10: 51
Data10: 52
Data10: 53
Data10: 54
Data10: 55
Data10: 56
Data10: 57
Data10: 58
Data10: 59
Data10: 60
Data10: 61
Data10: 62
Data10: 63
Data10: 64
Data10: 65
Data10: 66
Data10: 67
Data10: 68
Data10: 69
Data10: 70
Data10: 71
Data10: 72
Data10: 73
Data10: 74
Data10: 75
Data

In [5]:
print t2_out1.shape, t2_out2.shape, t2_out3.shape

(1751, 1792) (1751, 128) (1751, 128)


In [5]:
np.save('data/SFEW_2/Train/train2_features', train2_out)
np.save('data/SFEW_2/Train/train10_features', train10_out)
np.save('data/SFEW_2/Val/val_features', val_out)

In [8]:
X_train10_features = np.load('data/SFEW_2/Train/train10_features.npy')
X_train2_features = np.load('data/SFEW_2/Train/train2_features.npy')
Xvf = np.load('data/SFEW_2/Val/val_features.npy')

print X_train2_features.shape
print X_train10_features.shape
print Xvf.shape

(9801, 2048)
(1751, 2048)
(431, 2048)
